In [145]:
#Train, val, Test splitter
import os
from sklearn.model_selection import train_test_split
import shutil
import random

In [139]:
unsorted_data_dir = "raw_data/derivatives/"
split = (0.8,0.1,0.1)
random.seed(1234)
#__init__
for dirpath,_,files in os.walk("data/"):
    for run, scan in enumerate(sorted(files)):
        if scan.endswith(".nii.gz"): #maybe include labels?
            os.remove(os.path.join(dirpath, scan))

In [140]:


def get_all_data(unsorted_data_dir):
    data = []
    for dirpath,_,files in os.walk(unsorted_data_dir):
                for run, scan in enumerate(sorted(files)):
                    if scan.endswith(".nii.gz"):
                        data.append((run, os.path.join(dirpath, scan)))
    return data

def sort_by_subject(data):
    output = {sub: [] for sub in os.listdir(unsorted_data_dir)}
    for i in data:
        run,file = i
        output[file[len(unsorted_data_dir):len(unsorted_data_dir)+9]].append((run,file))
    return output

def sort_by_language(data):
    output = {"EN":[], "CN":[], "FR":[]}
    for i in data:
        run,file = i
        output[file[len(unsorted_data_dir)+4:len(unsorted_data_dir)+6]].append((run,file))
    return output

data = get_all_data(unsorted_data_dir)

In [144]:
print(len(data))

999


In [149]:
#config 1: same language, mixed subjects
language = "EN"
relevant_files = [i for _,i in sort_by_language(data)[language]]
random.shuffle(relevant_files)
split_ = (int(len(relevant_files)*split[0]), int(len(relevant_files)*split[0])+int(len(relevant_files)*split[1])) 
train = [f for f in relevant_files[:split_[0]]]
val = [f for f in relevant_files[split_[0]:split_[1]]]
test = [f for f in relevant_files[split_[1]:]]
folders = {"Train":train, "Val":val, "Test":test}
for k, v in folders.items():
    for file in v:
        shutil.copy(file, f"data/{k}/{language}")
    

In [ ]:
#config 2: same language, same subject

In [ ]:
#config 3: different language